In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
# Debug flags
run_voting = True
run_stacking = True
use_validation = True
analysis = use_validation and True

In [3]:
df = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')

In [4]:
if analysis:
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [5]:
if analysis:
    print(df.columns)

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')


In [6]:
df = df.drop(columns = ['PassengerId', 'Name'], axis=1)
if analysis:
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8492 non-null   object 
 1   CryoSleep     8476 non-null   object 
 2   Cabin         8494 non-null   object 
 3   Destination   8511 non-null   object 
 4   Age           8514 non-null   float64
 5   VIP           8490 non-null   object 
 6   RoomService   8512 non-null   float64
 7   FoodCourt     8510 non-null   float64
 8   ShoppingMall  8485 non-null   float64
 9   Spa           8510 non-null   float64
 10  VRDeck        8505 non-null   float64
 11  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(5)
memory usage: 755.7+ KB


In [7]:
#Checking out what each object contains in the columns where the data type within it are of type object.
planet = df['HomePlanet'].iloc[0]
cryosleep = df['CryoSleep'].iloc[0]
cabin = df['Cabin'].iloc[0]
destination = df['Destination'].iloc[0]
vip = df['VIP'].iloc[0]

#We see that cabin is of type string but contains useful information separated out by '/'

In [8]:
if 'Cabin' in df:
    df[['CabinDeck', 'CabinNumber', 'CabinSide']] = df['Cabin'].str.split('/', expand=True)
    df.drop(columns=['Cabin', 'CabinNumber'], inplace=True)
    
if analysis:
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8492 non-null   object 
 1   CryoSleep     8476 non-null   object 
 2   Destination   8511 non-null   object 
 3   Age           8514 non-null   float64
 4   VIP           8490 non-null   object 
 5   RoomService   8512 non-null   float64
 6   FoodCourt     8510 non-null   float64
 7   ShoppingMall  8485 non-null   float64
 8   Spa           8510 non-null   float64
 9   VRDeck        8505 non-null   float64
 10  Transported   8693 non-null   bool   
 11  CabinDeck     8494 non-null   object 
 12  CabinSide     8494 non-null   object 
dtypes: bool(1), float64(6), object(6)
memory usage: 823.6+ KB


In [9]:
#We use one-hot encoding on columns where the data are strings and cannot be converted to a true or false value
df = pd.get_dummies(df, columns=['HomePlanet', 'Destination', 'CabinDeck', 'CabinSide'], drop_first=False)

if analysis:
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CryoSleep                  8476 non-null   object 
 1   Age                        8514 non-null   float64
 2   VIP                        8490 non-null   object 
 3   RoomService                8512 non-null   float64
 4   FoodCourt                  8510 non-null   float64
 5   ShoppingMall               8485 non-null   float64
 6   Spa                        8510 non-null   float64
 7   VRDeck                     8505 non-null   float64
 8   Transported                8693 non-null   bool   
 9   HomePlanet_Earth           8693 non-null   bool   
 10  HomePlanet_Europa          8693 non-null   bool   
 11  HomePlanet_Mars            8693 non-null   bool   
 12  Destination_55 Cancri e    8693 non-null   bool   
 13  Destination_PSO J318.5-22  8693 non-null   bool 

In [10]:
#Imputing the data to ensure there are no missing values in the columns
#Starting with VIP and CryoSleep because they are true or false valued strings
df['CryoSleep'] = df['CryoSleep'].fillna(df['CryoSleep'].mode()[0] if not df['CryoSleep'].mode().empty else None)
df['VIP'] = df['VIP'].fillna(df['VIP'].mode()[0] if not df['VIP'].mode().empty else None)

#Lastly imputing all of the missing values for the columns of data type float 64
df['Age'] = df['Age'].fillna(df['Age'].mean())
df['RoomService'] = df['RoomService'].fillna(df['RoomService'].mean())
df['FoodCourt'] = df['FoodCourt'].fillna(df['FoodCourt'].mean())
df['ShoppingMall'] = df['ShoppingMall'].fillna(df['ShoppingMall'].mean())
df['Spa'] = df['Spa'].fillna(df['Spa'].mean())
df['VRDeck'] = df['VRDeck'].fillna(df['VRDeck'].mean())

#Converting every last column to a data type of float 64
df = df.astype('float64')
if analysis:
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CryoSleep                  8693 non-null   float64
 1   Age                        8693 non-null   float64
 2   VIP                        8693 non-null   float64
 3   RoomService                8693 non-null   float64
 4   FoodCourt                  8693 non-null   float64
 5   ShoppingMall               8693 non-null   float64
 6   Spa                        8693 non-null   float64
 7   VRDeck                     8693 non-null   float64
 8   Transported                8693 non-null   float64
 9   HomePlanet_Earth           8693 non-null   float64
 10  HomePlanet_Europa          8693 non-null   float64
 11  HomePlanet_Mars            8693 non-null   float64
 12  Destination_55 Cancri e    8693 non-null   float64
 13  Destination_PSO J318.5-22  8693 non-null   float

<ipython-input-10-83583dcd374f>:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['CryoSleep'] = df['CryoSleep'].fillna(df['CryoSleep'].mode()[0] if not df['CryoSleep'].mode().empty else None)
<ipython-input-10-83583dcd374f>:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['VIP'] = df['VIP'].fillna(df['VIP'].mode()[0] if not df['VIP'].mode().empty else None)


In [11]:
# Features and splits
from sklearn.model_selection import train_test_split
from sklearn import metrics
features = [column for column in df.columns if column != 'Transported']
X = df[features]
y = df['Transported']

In [12]:
# Splitting data
if use_validation:
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=1)
else:
    X_train, y_train = X, y

In [13]:
#Multilayer Perceptron(MLP)
from sklearn.neural_network import MLPClassifier

#Model with activation function of ReLu
modelRelu = MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu', solver='adam', max_iter=5000, random_state=42)
modelRelu.fit(X_train, y_train)

if analysis:
    #Prediction
    y_pred = modelRelu.predict(X_val)
    
    #Accuracy
    accuracy = metrics.accuracy_score(y_val, y_pred)
    confusion_matrix = metrics.confusion_matrix(y_val, y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    # Confusion matrix values
    TN, FP, FN, TP = confusion_matrix.ravel()
    print(f"Confusion matrix:\n"
          f"\t\tPredicted: 0\t\t\tPredicted: 1\n"
          f"Actual: 0\t{TN} (True Negative, TN)\t{FP} (False Positive, FP)\n"
          f"Actual: 1\t{FN} (False Negative, FN)\t{TP} (True Positive, TP)")

Accuracy: 0.79
Confusion matrix:
		Predicted: 0			Predicted: 1
Actual: 0	1048 (True Negative, TN)	243 (False Positive, FP)
Actual: 1	302 (False Negative, FN)	1015 (True Positive, TP)


In [14]:
#Model with activation function of ReLu with more hidden layers
modelRelu2 = MLPClassifier(hidden_layer_sizes=(256, 128, 64, 32), activation='relu', solver='adam', max_iter=5000, random_state=42)
modelRelu2.fit(X_train, y_train)

if analysis:
    #Prediction
    y_pred = modelRelu2.predict(X_val)
    
    #Accuracy
    accuracy = metrics.accuracy_score(y_val, y_pred)
    confusion_matrix = metrics.confusion_matrix(y_val, y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    # Confusion matrix values
    TN, FP, FN, TP = confusion_matrix.ravel()
    print(f"Confusion matrix:\n"
          f"\t\tPredicted: 0\t\t\tPredicted: 1\n"
          f"Actual: 0\t{TN} (True Negative, TN)\t{FP} (False Positive, FP)\n"
          f"Actual: 1\t{FN} (False Negative, FN)\t{TP} (True Positive, TP)")

Accuracy: 0.80
Confusion matrix:
		Predicted: 0			Predicted: 1
Actual: 0	985 (True Negative, TN)	306 (False Positive, FP)
Actual: 1	208 (False Negative, FN)	1109 (True Positive, TP)


In [15]:
#Model with activation function of Tanh
modelTanh = MLPClassifier(hidden_layer_sizes=(128, 64), activation='tanh', solver='adam', max_iter=5000, random_state=42)
modelTanh.fit(X_train, y_train)

if analysis:
    #Prediction
    y_pred = modelTanh.predict(X_val)
    
    #Accuracy
    accuracy = metrics.accuracy_score(y_val, y_pred)
    confusion_matrix = metrics.confusion_matrix(y_val, y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    # Confusion matrix values
    TN, FP, FN, TP = confusion_matrix.ravel()
    print(f"Confusion matrix:\n"
          f"\t\tPredicted: 0\t\t\tPredicted: 1\n"
          f"Actual: 0\t{TN} (True Negative, TN)\t{FP} (False Positive, FP)\n"
          f"Actual: 1\t{FN} (False Negative, FN)\t{TP} (True Positive, TP)")

Accuracy: 0.79
Confusion matrix:
		Predicted: 0			Predicted: 1
Actual: 0	983 (True Negative, TN)	308 (False Positive, FP)
Actual: 1	247 (False Negative, FN)	1070 (True Positive, TP)


In [16]:
#Model with activation function of Tanh with more hidden layers
modelTanh2 = MLPClassifier(hidden_layer_sizes=(256, 128, 64, 32), activation='tanh', solver='adam', max_iter=5000, random_state=42)
modelTanh2.fit(X_train, y_train)

if analysis:
    #Prediction
    y_pred = modelTanh2.predict(X_val)
    
    #Accuracy
    accuracy = metrics.accuracy_score(y_val, y_pred)
    confusion_matrix = metrics.confusion_matrix(y_val, y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    # Confusion matrix values
    TN, FP, FN, TP = confusion_matrix.ravel()
    print(f"Confusion matrix:\n"
          f"\t\tPredicted: 0\t\t\tPredicted: 1\n"
          f"Actual: 0\t{TN} (True Negative, TN)\t{FP} (False Positive, FP)\n"
          f"Actual: 1\t{FN} (False Negative, FN)\t{TP} (True Positive, TP)")

Accuracy: 0.79
Confusion matrix:
		Predicted: 0			Predicted: 1
Actual: 0	1025 (True Negative, TN)	266 (False Positive, FP)
Actual: 1	277 (False Negative, FN)	1040 (True Positive, TP)


In [17]:
#Model with activation function of logistic
modelLogistic = MLPClassifier(hidden_layer_sizes=(128, 64), activation='logistic', solver='adam', max_iter=5000, random_state=42)
modelLogistic.fit(X_train, y_train)

if analysis:
    #Prediction
    y_pred = modelLogistic.predict(X_val)
    
    #Accuracy
    accuracy = metrics.accuracy_score(y_val, y_pred)
    confusion_matrix = metrics.confusion_matrix(y_val, y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    # Confusion matrix values
    TN, FP, FN, TP = confusion_matrix.ravel()
    print(f"Confusion matrix:\n"
          f"\t\tPredicted: 0\t\t\tPredicted: 1\n"
          f"Actual: 0\t{TN} (True Negative, TN)\t{FP} (False Positive, FP)\n"
          f"Actual: 1\t{FN} (False Negative, FN)\t{TP} (True Positive, TP)")

Accuracy: 0.78
Confusion matrix:
		Predicted: 0			Predicted: 1
Actual: 0	1080 (True Negative, TN)	211 (False Positive, FP)
Actual: 1	350 (False Negative, FN)	967 (True Positive, TP)


In [18]:
#Model with activation function of logistic with more hidden layers
modelLogistic2 = MLPClassifier(hidden_layer_sizes=(256, 128, 64, 32), activation='logistic', solver='adam', max_iter=5000, random_state=42)
modelLogistic2.fit(X_train, y_train)

if analysis:
    #Prediction
    y_pred = modelLogistic2.predict(X_val)
    
    #Accuracy
    accuracy = metrics.accuracy_score(y_val, y_pred)
    confusion_matrix = metrics.confusion_matrix(y_val, y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    #print("Confusion matrix:\n", confusion_matrix)
    # Confusion matrix values
    TN, FP, FN, TP = confusion_matrix.ravel()
    print(f"Confusion matrix:\n"
          f"\t\tPredicted: 0\t\t\tPredicted: 1\n"
          f"Actual: 0\t{TN} (True Negative, TN)\t{FP} (False Positive, FP)\n"
          f"Actual: 1\t{FN} (False Negative, FN)\t{TP} (True Positive, TP)")

Accuracy: 0.78
Confusion matrix:
		Predicted: 0			Predicted: 1
Actual: 0	1030 (True Negative, TN)	261 (False Positive, FP)
Actual: 1	313 (False Negative, FN)	1004 (True Positive, TP)


In [19]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)

if analysis:
    y_pred = nb_classifier.predict(X_val)
    
    accuracy = accuracy_score(y_val, y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    print("Classification Report:\n", classification_report(y_val, y_pred))

     # Confusion matrix values
    confusion_matrix = metrics.confusion_matrix(y_val, y_pred)
    TN, FP, FN, TP = confusion_matrix.ravel()
    print(f"Confusion matrix:\n"
          f"\t\tPredicted: 0\t\t\tPredicted: 1\n"
          f"Actual: 0\t{TN} (True Negative, TN)\t{FP} (False Positive, FP)\n"
          f"Actual: 1\t{FN} (False Negative, FN)\t{TP} (True Positive, TP)")

Accuracy: 0.74
Classification Report:
               precision    recall  f1-score   support

         0.0       0.85      0.58      0.69      1291
         1.0       0.69      0.90      0.78      1317

    accuracy                           0.74      2608
   macro avg       0.77      0.74      0.74      2608
weighted avg       0.77      0.74      0.74      2608

Confusion matrix:
		Predicted: 0			Predicted: 1
Actual: 0	752 (True Negative, TN)	539 (False Positive, FP)
Actual: 1	129 (False Negative, FN)	1188 (True Positive, TP)


In [20]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, StratifiedKFold
import numpy as np

# Initialize SVM classifier
svm_rbf_classifier = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svm_rbf_classifier.fit(X_train, y_train)

if analysis:
    y_pred = svm_rbf_classifier.predict(X_val)
    
    accuracy = accuracy_score(y_val, y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    print("Classification Report:\n", classification_report(y_val, y_pred))

 # Confusion matrix values
    confusion_matrix = metrics.confusion_matrix(y_val, y_pred)
    TN, FP, FN, TP = confusion_matrix.ravel()
    print(f"Confusion matrix:\n"
          f"\t\tPredicted: 0\t\t\tPredicted: 1\n"
          f"Actual: 0\t{TN} (True Negative, TN)\t{FP} (False Positive, FP)\n"
          f"Actual: 1\t{FN} (False Negative, FN)\t{TP} (True Positive, TP)")

Accuracy: 0.79
Classification Report:
               precision    recall  f1-score   support

         0.0       0.85      0.71      0.77      1291
         1.0       0.75      0.88      0.81      1317

    accuracy                           0.79      2608
   macro avg       0.80      0.79      0.79      2608
weighted avg       0.80      0.79      0.79      2608

Confusion matrix:
		Predicted: 0			Predicted: 1
Actual: 0	912 (True Negative, TN)	379 (False Positive, FP)
Actual: 1	159 (False Negative, FN)	1158 (True Positive, TP)


In [21]:
# Initialize sigmoid SVM classifier
svm_sig_classifier = SVC(kernel='sigmoid', coef0=0.5, random_state=42)
svm_sig_classifier.fit(X_train, y_train)

if analysis:
    y_pred = svm_sig_classifier.predict(X_val)
    
    accuracy = accuracy_score(y_val, y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    print("Classification Report:\n", classification_report(y_val, y_pred))
    
    # Confusion matrix values
    confusion_matrix = metrics.confusion_matrix(y_val, y_pred)
    TN, FP, FN, TP = confusion_matrix.ravel()
    print(f"Confusion matrix:\n"
          f"\t\tPredicted: 0\t\t\tPredicted: 1\n"
          f"Actual: 0\t{TN} (True Negative, TN)\t{FP} (False Positive, FP)\n"
          f"Actual: 1\t{FN} (False Negative, FN)\t{TP} (True Positive, TP)")

Accuracy: 0.73
Classification Report:
               precision    recall  f1-score   support

         0.0       0.73      0.73      0.73      1291
         1.0       0.73      0.73      0.73      1317

    accuracy                           0.73      2608
   macro avg       0.73      0.73      0.73      2608
weighted avg       0.73      0.73      0.73      2608

Confusion matrix:
		Predicted: 0			Predicted: 1
Actual: 0	940 (True Negative, TN)	351 (False Positive, FP)
Actual: 1	350 (False Negative, FN)	967 (True Positive, TP)


In [22]:
# Decision Tree Models
# A ton of code was referenced from https://www.datacamp.com/tutorial/decision-tree-classification-python
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

dt_classifier = DecisionTreeClassifier(criterion='entropy', max_depth=5)
dt_classifier = dt_classifier.fit(X_train, y_train)

if analysis:
    y_pred = dt_classifier.predict(X_val)
    
    accuracy = accuracy_score(y_val, y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    print("Classification Report:\n", classification_report(y_val, y_pred))
    
    # Confusion matrix values
    confusion_matrix = metrics.confusion_matrix(y_val, y_pred)
    TN, FP, FN, TP = confusion_matrix.ravel()
    print(f"Confusion matrix:\n"
          f"\t\tPredicted: 0\t\t\tPredicted: 1\n"
          f"Actual: 0\t{TN} (True Negative, TN)\t{FP} (False Positive, FP)\n"
          f"Actual: 1\t{FN} (False Negative, FN)\t{TP} (True Positive, TP)")

Accuracy: 0.78
Classification Report:
               precision    recall  f1-score   support

         0.0       0.86      0.65      0.74      1291
         1.0       0.73      0.89      0.80      1317

    accuracy                           0.78      2608
   macro avg       0.79      0.77      0.77      2608
weighted avg       0.79      0.78      0.77      2608

Confusion matrix:
		Predicted: 0			Predicted: 1
Actual: 0	845 (True Negative, TN)	446 (False Positive, FP)
Actual: 1	139 (False Negative, FN)	1178 (True Positive, TP)


In [23]:
# Random Forest models
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=13, random_state=1)
rf_classifier.fit(X_train, y_train)

if analysis:
    y_pred = rf_classifier.predict(X_val)
    
    accuracy = accuracy_score(y_val, y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    print("Classification Report:\n", classification_report(y_val, y_pred))
    
    # Confusion matrix values
    confusion_matrix = metrics.confusion_matrix(y_val, y_pred)
    TN, FP, FN, TP = confusion_matrix.ravel()
    print(f"Confusion matrix:\n"
          f"\t\tPredicted: 0\t\t\tPredicted: 1\n"
          f"Actual: 0\t{TN} (True Negative, TN)\t{FP} (False Positive, FP)\n"
          f"Actual: 1\t{FN} (False Negative, FN)\t{TP} (True Positive, TP)")

Accuracy: 0.79
Classification Report:
               precision    recall  f1-score   support

         0.0       0.78      0.82      0.80      1291
         1.0       0.81      0.77      0.79      1317

    accuracy                           0.79      2608
   macro avg       0.80      0.79      0.79      2608
weighted avg       0.80      0.79      0.79      2608

Confusion matrix:
		Predicted: 0			Predicted: 1
Actual: 0	1054 (True Negative, TN)	237 (False Positive, FP)
Actual: 1	299 (False Negative, FN)	1018 (True Positive, TP)


In [24]:
# Ensemble Method
models = [
    ('2-layer Relu Neural Network', modelRelu),
    ('4-layer Relu Neural Network', modelRelu2),
    ('2-layer Tanh Neural Network', modelTanh),
    ('4-layer Tanh Neural Network', modelTanh2),
    ('2-layer Logistic Neural Network', modelLogistic),
    ('4-layer Logistic Neural Network', modelLogistic2),
    ('Naive Bayes Classifier', nb_classifier), 
    ('SVM Sigmoid Classifier', svm_sig_classifier), 
    ('SVM RBF Classifier', svm_rbf_classifier), 
    ('Decision Tree Classifier', dt_classifier), 
    ('Random Forest Classifier', rf_classifier)
]

In [25]:
from sklearn.ensemble import VotingClassifier

if run_voting:
    voting_classifier = VotingClassifier(estimators=models, voting='hard')
    voting_classifier.fit(X_train, y_train)
    
    if analysis:
        y_pred = voting_classifier.predict(X_val)
        
        accuracy = accuracy_score(y_val, y_pred)
        print(f"Accuracy: {accuracy:.2f}")
        print("Classification Report:\n", classification_report(y_val, y_pred))
        
        # Confusion matrix values
        confusion_matrix = metrics.confusion_matrix(y_val, y_pred)
        TN, FP, FN, TP = confusion_matrix.ravel()
        print(f"Confusion matrix:\n"
              f"\t\tPredicted: 0\t\t\tPredicted: 1\n"
              f"Actual: 0\t{TN} (True Negative, TN)\t{FP} (False Positive, FP)\n"
              f"Actual: 1\t{FN} (False Negative, FN)\t{TP} (True Positive, TP)")

Accuracy: 0.81
Classification Report:
               precision    recall  f1-score   support

         0.0       0.82      0.78      0.80      1291
         1.0       0.79      0.84      0.82      1317

    accuracy                           0.81      2608
   macro avg       0.81      0.81      0.81      2608
weighted avg       0.81      0.81      0.81      2608

Confusion matrix:
		Predicted: 0			Predicted: 1
Actual: 0	1004 (True Negative, TN)	287 (False Positive, FP)
Actual: 1	213 (False Negative, FN)	1104 (True Positive, TP)


In [26]:
from sklearn.ensemble import StackingClassifier

if run_stacking:
    stacking_classifier = StackingClassifier(estimators=models)
    stacking_classifier.fit(X_train, y_train)
    
    if analysis:
        y_pred = stacking_classifier.predict(X_val)
        
        accuracy = accuracy_score(y_val, y_pred)
        print(f"Accuracy: {accuracy:.2f}")
        print("Classification Report:\n", classification_report(y_val, y_pred))
        
        # Confusion matrix values
        confusion_matrix = metrics.confusion_matrix(y_val, y_pred)
        TN, FP, FN, TP = confusion_matrix.ravel()
        print(f"Confusion matrix:\n"
              f"\t\tPredicted: 0\t\t\tPredicted: 1\n"
              f"Actual: 0\t{TN} (True Negative, TN)\t{FP} (False Positive, FP)\n"
              f"Actual: 1\t{FN} (False Negative, FN)\t{TP} (True Positive, TP)")

Accuracy: 0.80
Classification Report:
               precision    recall  f1-score   support

         0.0       0.80      0.80      0.80      1291
         1.0       0.81      0.81      0.81      1317

    accuracy                           0.80      2608
   macro avg       0.80      0.80      0.80      2608
weighted avg       0.80      0.80      0.80      2608

Confusion matrix:
		Predicted: 0			Predicted: 1
Actual: 0	1036 (True Negative, TN)	255 (False Positive, FP)
Actual: 1	256 (False Negative, FN)	1061 (True Positive, TP)


In [27]:
df = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
if analysis:
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [28]:
# On actual test data
df = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
submission_id = df.PassengerId

df = df.drop(columns = ['PassengerId', 'Name'], axis=1)

# Creating new features for deck and cabin side
df[['CabinDeck', 'CabinNumber', 'CabinSide']] = df["Cabin"].str.split("/", expand=True)
df = df.drop(columns = ["Cabin", "CabinNumber"], axis=1)

#Imputing the data to ensure there are no missing values in the columns
#Starting with VIP and CryoSleep because they are true or false valued strings
df['CryoSleep'] = df['CryoSleep'].fillna(df['CryoSleep'].mode()[0] if not df['CryoSleep'].mode().empty else None)
df['VIP'] = df['VIP'].fillna(df['VIP'].mode()[0] if not df['VIP'].mode().empty else None)

#We use one-hot encoding on columns where the data are strings and cannot be converted to a true or false value
df = pd.get_dummies(df, columns=['HomePlanet', 'Destination', 'CabinDeck', 'CabinSide'], drop_first=False)

#Lastly imputing all of the missing values for the columns of data type float 64
df['Age'] = df['Age'].fillna(df['Age'].mean())
df['RoomService'] = df['RoomService'].fillna(df['RoomService'].mean())
df['FoodCourt'] = df['FoodCourt'].fillna(df['FoodCourt'].mean())
df['ShoppingMall'] = df['ShoppingMall'].fillna(df['ShoppingMall'].mean())
df['Spa'] = df['Spa'].fillna(df['Spa'].mean())
df['VRDeck'] = df['VRDeck'].fillna(df['VRDeck'].mean())

#Converting every last column to a data type of float 64
df = df.astype('float64')

features = [column for column in df.columns if column != 'Transported']
X_train = df[features]

# Get the predictions for testdata
if run_voting:
    predictions = voting_classifier.predict(X_train)
elif run_stacking:
    predictions = stacking_classifier.predict(X_train)
    
n_predictions = (predictions > 0.5).astype(bool)
output = pd.DataFrame({'PassengerId': submission_id,
                       'Transported': n_predictions.squeeze()})

output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

<ipython-input-28-f6e24bdb0062>:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['CryoSleep'] = df['CryoSleep'].fillna(df['CryoSleep'].mode()[0] if not df['CryoSleep'].mode().empty else None)
<ipython-input-28-f6e24bdb0062>:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['VIP'] = df['VIP'].fillna(df['VIP'].mode()[0] if not df['VIP'].mode().empty else None)


Your submission was successfully saved!
